# Planning Permits Analysis

This analysis uses data from https://discover.data.vic.gov.au/dataset?q=planning%20permits.


'The Victorian Building Authority (VBA) collects information from building surveyors on the number, value and type of building permits issued each month as part of its functions under the Building Act 1993'
 
Firstly there is also a building permit activity monthly summary dataset which has been updated to April 2021 (https://discover.data.vic.gov.au/dataset/building-permit-activity-monthly-summaries). This is an aggregated dataset which includes data visualisations which track building use, costs, 


The summary dataset aggregates data from separate annual datesets which run from to 2020 (https://discover.data.vic.gov.au/dataset/building-permit-activity-data-2020). ** up to 2021 on building vic autority (VBA) site ** Within these datasets, each record or row represents a single permit.

These annual datasets include over 40 pieces of information per record, such as details of what is to be built (or demolished),the intended use of the building, the ownership sector, and the building costs. 

In addition, the location of the building can be viewed down to the street name level, with postal codes, suburbs and regions also included.

A comprehensive data dictionary for the building permit datsets can be found on the VBA site here (https://www.vba.vic.gov.au/about/data), as well as a detailed data quality statement PDF which includes clear summaries about what the data represents.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# VBA DataVic data, hosted on VBA website
dataset_urls = {
    '2021':'https://www.vba.vic.gov.au/__data/assets/excel_doc/0004/143572/VBA-DataVic-Building-Permits-2021.xlsx',
    '2020':'https://www.vba.vic.gov.au/__data/assets/file/0012/110028/VBA-DataVic-Building-Permits-2020.xlsb',
    '2019':'https://www.vba.vic.gov.au/__data/assets/file/0015/103515/VBA-DataVic-Building-Permits-2019.xlsb'
}

dfs = {}

for dataset in dataset_urls:
    if dataset_urls[dataset].endswith('xlsb'):
        # to read xlsb, you need to install pyxlsb using pip at command prompt (pip install pyxlsb)
        dfs[dataset] = pd.read_excel(dataset_urls[dataset],sheet_name=1,engine='pyxlsb') 
    else:
        dfs[dataset] = pd.read_excel(dataset_urls[dataset],sheet_name=1) 

In [3]:
# Do the datasets contain the same number of variables?
for year in dfs:
    print(f'{year}: {len(dfs[year].columns)}')
    
len(dfs['2019'].columns)== len(dfs['2020'].columns) == len(dfs['2021'].columns)

2021: 37
2020: 37
2019: 41


False

In [4]:
# The 2019 dataset has more variables than the subsequent years; let's come back to 2019 

# First, let's check that the 37 variables in the 2020 and 2021 datasets are the same

for i,assertion in enumerate(dfs['2020'].columns == dfs['2021'].columns):
    if assertion == False:
        print(f"Year  Mis-matched column")
        for year in ['2020','2021']:
            print(f'{year}: {dfs[year].columns[i]}')


Year  Mis-matched column
2020: BASIS_ BCA
2021: BASIS_BCA


In [5]:
# The column 'BASIS_BCA' is mis-spelt in the 2020 dataset; let's correct that
dfs['2020'].rename(columns={'BASIS_ BCA':'BASIS_BCA'},inplace=True)

In [6]:
# let's confirm that's fixed now:
for i,assertion in enumerate(dfs['2020'].columns == dfs['2021'].columns):
    if assertion == False:
        print(f"Year  Mis-matched column")
        for year in ['2020','2021']:
            print(f'{year}: {dfs[year].columns[i]}')

# yes, all good!

In [7]:
# Let's look at the 2019 and 2020 variables
print(dfs['2019'].columns)
print(dfs['2020'].columns)


Index(['permit_stage_number', 'permit_date', 'BASIS_Month_Y', 'BASIS_Month_M',
       'Reported_Levy_amount', 'Calculated_Levy_amount',
       'Reported_Cost_of_works', 'Site_street', 'Site_suburb', 'site_pcode',
       'Municipal Name', 'Municipal Full Name', 'Region', 'Sub_Region',
       'Sub_Region1', 'Allotment_Area', 'Builder_suburb', 'Builder_state',
       'Builder_pcode', 'Material_Code_Floor', 'Material_Code_Frame',
       'Material_Code_Roof', 'Material_Code_Walls', 'dwellings_before_work',
       'dwellings_after_work', 'Number_of_storeys', 'number_demolished',
       'Floor_area', 'Multiple_Dwellings', 'cost_of_works_domestic',
       'Permit_app_date', 'BACV_applicable_flag', 'Calculated_levy_BACV',
       'solar_hot_water', 'rainwater_tank', 'est_cost_project',
       'BASIS_Building_Use', 'BASIS_NOW', 'BASIS_BCA', 'BASIS_OwnershipSector',
       'BASIS_OwnerBuilder'],
      dtype='object')
Index(['permit_stage_number', 'permit_date', 'BASIS_Month_Y', 'BASIS_Month_M',
  

The 2019 variables are in a different order, and spelt differently.

To determine how to proceed, lets compare

- The VBA data dictonary (last modified 2015, at time of writing)
- The 2019 dataset columns
- The consolidated 2020/21 columns (following space correction of BASIS_BCA variable name, above)
- A combined proposed plain text variable name without special characters 

| ID | Data dictionary        |          2019          |               2020/21               | Proposed variable name        |
|----|------------------------|:----------------------:|:-----------------------------------:|-------------------------------|
| 1  | permit_stage_number    | permit_stage_number    | permit_stage_number                 | Permit Stage Number           |
| 2  | permit_date            | permit_date            | permit_date                         | Permit Date                   |
| 3  | BASIS_Month_Y          | BASIS_Month_Y          | BASIS_Month_Y                       | Year                          |
| 4  | BASIS_Month_M          | BASIS_Month_M          | BASIS_Month_M                       | Month                         |
| 5  | Reported_Levy_amount   | Reported_Levy_amount   |                                     | Reported Levy Amount          |
| 6  | Calculated_Levy_amount | Calculated_Levy_amount |                                     | Calculated Levy Amount        |
| 7  |                        |                        | Original_Levy_Paid__c               | Original Levy Paid            |
| 8  | Reported_Cost_of_works | Reported_Cost_of_works | Reported_Cost_of_works              | Reported Cost Of Works        |
| 9  | Site_street            | Site_street            | site_street_name__c                 | Site Street                   |
| 10 | Site_suburb            | Site_suburb            | site_town_suburb__c                 | Site Suburb                   |
| 11 | site_pcode             | site_pcode             | site_postcode__c                    | Site Postcode                 |
| 12 | Municipal name         | Municipal Name         | Site_Municipality                   | Municipal Name                |
| 13 | Municipal full name    | Municipal Full Name    | Municipal Full Name                 | Municipal Full Name           |
| 14 | Region                 | Region                 | Region                              | Region                        |
| 15 | Sub_Region             | Sub_Region             | Sub_Region                          | Sub Region                    |
| 16 | Sub_Region1            | Sub_Region1            | Sub_Region1                         | Sub Region1                   |
| 17 | Allotment_Area         | Allotment_Area         | Allotment_Area__c                   | Allotment Area                |
| 18 | Builder_suburb         | Builder_suburb         | Builder_Town_Suburb__c              | Builder Suburb                |
| 19 | Builder_state          | Builder_state          | Builder_State__c                    | Builder State                 |
| 20 | Builder_pcode          | Builder_pcode          | Builder_Postcode__c                 | Builder Postcode              |
| 21 | Material_Code_Floor    | Material_Code_Floor    | Floor_Material__c                   | Material Code Floor           |
| 22 | Material_Code_Frame    | Material_Code_Frame    | Frame_Material__c                   | Material Code Frame           |
| 23 | Material_Code_Roof     | Material_Code_Roof     | Roof_Cladding_Material__c           | Material Code Roof            |
| 24 | Material_Code_Walls    | Material_Code_Walls    | External_Wall_Material__c           | Material Code Walls           |
| 25 | dwellings_before_work  | dwellings_before_work  | Number_of_Existing_Dwellings__c     | Existing Dwellings            |
| 26 | dwellings_after_work   | dwellings_after_work   | Number_of_New_Dwellings__c          | New Dwellings                 |
| 27 | Number_of_storeys      | Number_of_storeys      | Number_of_Storeys__c                | Storeys                       |
| 28 | number_demolished      | number_demolished      | Number_of_Dwellings_Demolished__c   | Dwellings Demolished          |
| 29 | Floor_area             | Floor_area             | Total_Floor_Area__c                 | Floor Area                    |
| 30 | Multiple_Dwellings     | Multiple_Dwellings     |                                     | Multiple Dwellings            |
| 31 | cost_of_works_domestic | cost_of_works_domestic |                                     | Cost Of Works Domestic        |
| 32 | Permit_app_date        | Permit_app_date        | Building_Permit_Application_Date__c | Permit Application Date       |
| 33 | BACV_applicable_flag   | BACV_applicable_flag   |                                     | BACV Applicable Flag          |
| 34 | Calculated_levy_BACV   | Calculated_levy_BACV   |                                     | Calculated Levy BACV          |
| 35 |                        |                        | DBDRV Levy                          | DBDRV Levy                    |
| 36 | solar_hot_water        | solar_hot_water        | Solar_Hot_Water_Indicator__c        | Solar Hot Water               |
| 37 | rainwater_tank         | rainwater_tank         | Rainwater_Tank_Indicator__c         | Rainwater Tank                |
| 38 | est_cost_project       | est_cost_project       | Total_Estimated_Cost_of_Works__c    | Total Estimated Cost of Works |
| 39 | BASIS_Zone             | BASIS_Building_Use     | BASIS_Building_Use                  | BASIS Building Use            |
| 40 | BASIS_NOW              | BASIS_NOW              | BASIS_NOW                           | BASIS NOW                     |
| 41 | BASIS_BCA              | BASIS_BCA              | BASIS_BCA                           | BASIS BCA                     |
| 42 | BASIS_OwnershipSector  | BASIS_OwnershipSector  | BASIS_Ownership_Sector              | BASIS Ownership Sector        |
| 43 | BASIS_OwnerBuilder     | BASIS_OwnerBuilder     | BASIS_Owner_Builder                 | BASIS Owner Builder           |

Through this comparison it is apparent that,

- The 2019 dataset is mostly in accord with the data dictionary
- The data dictionary variable 'BASIS_ZONE' appears to have been framed as 'BASIS_NOW' in the 2019, 2020 and 2021 datasets
- Reported and calculated levy amounts are not recorded in 2020/21; there was an original levy paid variable instead
- Multiple dwellings and domestic cost of works wa not recorded recorded in 2020/21
- The BACV applicable flag and calculated levy were not recorded in 2020/21; there is however a 'DBDRV Levy'

Now we will ensure that each year shares the same variables and names, with missing values where these variables do not directly correspond, as per the proposed variable names in the table above.

In [8]:
# create columns which did not exist with null values
dfs['2019']['Original Levy Paid'] = np.nan
dfs['2019']['DBDRV Levy'] = np.nan
for year in ['2020','2021']:
    dfs[year]['Reported Levy Amount'] = np.nan
    dfs[year]['Calculated Levy Amount'] = np.nan
    dfs[year]['Multiple Dwellings'] = np.nan
    dfs[year]['Cost Of Works Domestic'] = np.nan
    dfs[year]['BACV Applicable Flag'] = np.nan
    dfs[year]['Calculated Levy BACV'] = np.nan

# rename columns
rename_2019_to_proposed = {'permit_stage_number':'Permit Stage Number','permit_date':'Permit Date','BASIS_Month_Y':'Year','BASIS_Month_M':'Month','Reported_Levy_amount':'Reported Levy Amount','Calculated_Levy_amount':'Calculated Levy Amount','Reported_Cost_of_works':'Reported Cost Of Works','Site_street':'Site Street','Site_suburb':'Site Suburb','site_pcode':'Site Postcode','Municipal Name':'Municipal Name','Municipal Full Name':'Municipal Full Name','Region':'Region','Sub_Region':'Sub Region','Sub_Region1':'Sub Region1','Allotment_Area':'Allotment Area','Builder_suburb':'Builder Suburb','Builder_state':'Builder State','Builder_pcode':'Builder Postcode','Material_Code_Floor':'Material Code Floor','Material_Code_Frame':'Material Code Frame','Material_Code_Roof':'Material Code Roof','Material_Code_Walls':'Material Code Walls','dwellings_before_work':'Existing Dwellings','dwellings_after_work':'New Dwellings','Number_of_storeys':'Storeys','number_demolished':'Dwellings Demolished','Floor_area':'Floor Area','Multiple_Dwellings':'Multiple Dwellings','cost_of_works_domestic':'Cost Of Works Domestic','Permit_app_date':'Permit Application Date','BACV_applicable_flag':'BACV Applicable Flag','Calculated_levy_BACV':'Calculated Levy BACV','solar_hot_water':'Solar Hot Water','rainwater_tank':'Rainwater Tank','est_cost_project':'Total Estimated Cost of Works','BASIS_Building_Use':'BASIS Building Use','BASIS_NOW':'BASIS NOW','BASIS_BCA':'BASIS BCA','BASIS_OwnershipSector':'BASIS Ownership Sector','BASIS_OwnerBuilder':'BASIS Owner Builder'}
rename_202x_to_proposed = {'permit_stage_number':'Permit Stage Number','permit_date':'Permit Date','BASIS_Month_Y':'Year','BASIS_Month_M':'Month','Original_Levy_Paid__c':'Original Levy Paid','Reported_Cost_of_works':'Reported Cost Of Works','site_street_name__c':'Site Street','site_town_suburb__c':'Site Suburb','site_postcode__c':'Site Postcode','Site_Municipality':'Municipal Name','Municipal Full Name':'Municipal Full Name','Region':'Region','Sub_Region':'Sub Region','Sub_Region1':'Sub Region1','Allotment_Area__c':'Allotment Area','Builder_Town_Suburb__c':'Builder Suburb','Builder_State__c':'Builder State','Builder_Postcode__c':'Builder Postcode','Floor_Material__c':'Material Code Floor','Frame_Material__c':'Material Code Frame','Roof_Cladding_Material__c':'Material Code Roof','External_Wall_Material__c':'Material Code Walls','Number_of_Existing_Dwellings__c':'Existing Dwellings','Number_of_New_Dwellings__c':'New Dwellings','Number_of_Storeys__c':'Storeys','Number_of_Dwellings_Demolished__c':'Dwellings Demolished','Total_Floor_Area__c':'Floor Area','Building_Permit_Application_Date__c':'Permit Application Date','DBDRV Levy':'DBDRV Levy','Solar_Hot_Water_Indicator__c':'Solar Hot Water','Rainwater_Tank_Indicator__c':'Rainwater Tank','Total_Estimated_Cost_of_Works__c':'Total Estimated Cost of Works','BASIS_Building_Use':'BASIS Building Use','BASIS_NOW':'BASIS NOW','BASIS_BCA':'BASIS BCA','BASIS_Ownership_Sector':'BASIS Ownership Sector','BASIS_Owner_Builder':'BASIS Owner Builder'}

dfs['2019'].rename(columns = rename_2019_to_proposed,inplace=True)
dfs['2020'].rename(columns = rename_202x_to_proposed,inplace=True)
dfs['2021'].rename(columns = rename_202x_to_proposed,inplace=True)

# order columns
columns = ['Permit Stage Number','Permit Date','Year','Month','Reported Levy Amount','Calculated Levy Amount','Original Levy Paid','Reported Cost Of Works','Site Street','Site Suburb','Site Postcode','Municipal Name','Municipal Full Name','Region','Sub Region','Sub Region1','Allotment Area','Builder Suburb','Builder State','Builder Postcode','Material Code Floor','Material Code Frame','Material Code Roof','Material Code Walls','Existing Dwellings','New Dwellings','Storeys','Dwellings Demolished','Floor Area','Multiple Dwellings','Cost Of Works Domestic','Permit Application Date','BACV Applicable Flag','Calculated Levy BACV','DBDRV Levy','Solar Hot Water','Rainwater Tank','Total Estimated Cost of Works','BASIS Building Use','BASIS NOW','BASIS BCA','BASIS Ownership Sector','BASIS Owner Builder']
for year in ['2019','2020','2021']:
    dfs[year] = dfs[year][columns]
    


In [18]:
# Confirm that the year variable correctly indexes each year, and year is not missing
# This is important for when we join these seperate datasets to ensure they can be 
# correctly distinguished
years = ['2019','2020','2021']
for year in ['2019','2020','2021']:
    print(f'\n{year}')
    print(dfs[year]['Year'].describe())
    print(f'Missing year: {dfs[year]["Year"].isna().sum()}')



2019
count    101988.0
mean       2019.0
std           0.0
min        2019.0
25%        2019.0
50%        2019.0
75%        2019.0
max        2019.0
Name: Year, dtype: float64
Missing year: 0

2020
count    113430.0
mean       2020.0
std           0.0
min        2020.0
25%        2020.0
50%        2020.0
75%        2020.0
max        2020.0
Name: Year, dtype: float64
Missing year: 0

2021
count    97662.0
mean      2021.0
std          0.0
min       2021.0
25%       2021.0
50%       2021.0
75%       2021.0
max       2021.0
Name: Year, dtype: float64
Missing year: 0


In [19]:
df = pd.concat(dfs)

In [23]:
# Describe the combined data:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Permit Stage Number,313080.0,7.157372e-01,1.621243e+02,0.0,0.000,0.0,0.0000,6.989000e+04
Year,313080.0,2.019986e+03,7.984407e-01,2019.0,2019.000,2020.0,2021.0000,2.021000e+03
Month,313080.0,6.137176e+00,3.195185e+00,1.0,3.000,6.0,9.0000,1.200000e+01
Reported Levy Amount,101988.0,4.732104e+02,2.493533e+03,0.0,20.200,153.6,384.0000,1.996800e+05
Calculated Levy Amount,53026.0,4.899761e+02,2.719822e+03,0.0,20.096,140.8,385.2375,1.996800e+05
Original Levy Paid,211092.0,8.607468e+02,9.410323e+03,0.0,21.120,192.0,387.8400,1.291647e+06
Reported Cost Of Works,313080.0,3.597323e+05,1.992675e+06,0.0,16060.000,145000.0,302058.5000,3.838709e+08
Allotment Area,279273.0,8.291374e+04,9.350639e+06,0.0,263.000,531.0,858.0000,4.265539e+09
Material Code Floor,272076.0,1.921145e+01,1.685988e+01,0.0,20.000,20.0,20.0000,4.020000e+03
Material Code Frame,277050.0,3.722993e+01,2.013500e+01,0.0,40.000,40.0,40.0000,9.000000e+01


In [ ]:
df